In [26]:
import json
import os
from io import BytesIO

import folium
import geopandas as gpd
import numpy as np
import osmnx as ox
import pandas as pd
import rasterio
import requests
from owslib.wfs import WebFeatureService
from pyproj import CRS
from rasterio.transform import from_bounds
from rasterstats import zonal_stats
from shapely.geometry import Point, box, shape

def list_wfs_layers(wfs_url):
    wfs = WebFeatureService(url=wfs_url, version='2.0.0')
    print("Service title:", wfs.identification.title)
    for name, md in wfs.contents.items():
        print(f"- {name} :: {getattr(md, 'title', '')}")

list_wfs_layers("https://geo.api.vlaanderen.be/Gebouwenregister/wfs")

OUT_DIR = "data_leuven"
TARGET_EPSG = 31370
WGS84 = 4326

GRB_WFS_URL = "https://geo.api.vlaanderen.be/Gebouwenregister/wfs"
GRB_WFS_LAYER = "Gebouwenregister:Gebouw"

LARGE_ROOF_MIN_M2 = 500
FILL_FACTOR = 0.6
KWH_PER_M2 = 200.0
CO2_KG_PER_KWH = 0.23

# -+-+Grid capacity for congestion calculation
TOTAL_GRID_CAPACITY_KW = 130000  # Total grid capacity in kW

def get_leuven_boundary(place="Leuven, Belgium", epsg=TARGET_EPSG):
    import osmnx as ox
    g = ox.geocode_to_gdf(place).to_crs(epsg)
    return g.dissolve().reset_index(drop=True)

def fetch_buildings_wfs(leuven_gdf, url, layer, epsg=TARGET_EPSG):
    wfs = WebFeatureService(url=url, version="1.1.0")
    minx, miny, maxx, maxy = leuven_gdf.total_bounds
    resp = wfs.getfeature(typename=layer,
                          bbox=(minx, miny, maxx, maxy, f"EPSG:{epsg}"),
                          outputFormat="json")
    gdf = gpd.read_file(BytesIO(resp.read())).to_crs(epsg)
    return gdf

def clean_columns(g):
    g = g.copy()
    ren = {}
    if "id" in g.columns: ren["id"] = "src_id"
    if "Id" in g.columns: ren["Id"] = "wfs_id"
    if "ObjectId" in g.columns: ren["ObjectId"] = "object_id"
    g = g.rename(columns=ren)
    if "VersieId" in g.columns:
        g["VersieId"] = pd.to_datetime(g["VersieId"], utc=True, errors="coerce").dt.strftime("%Y-%m-%d %H:%M:%S")
    g = g[g.geometry.notnull() & g.geometry.is_valid]
    return g

def add_area_centroid(g):
    g = g.copy()
    g["area_m2"] = g.geometry.area
    cen = g.geometry.centroid
    g["centroid_x"] = cen.x
    g["centroid_y"] = cen.y
    return g

def make_geojson_safe(g):
    g = g.copy().drop(columns=["VersieId"], errors="ignore")
    for c in g.columns:
        if c != "geometry" and pd.api.types.is_datetime64_any_dtype(g[c]):
            g[c] = pd.to_datetime(g[c], errors="coerce").dt.strftime("%Y-%m-%d %H:%M:%S")
    return g

def save_layers(gdf, name, out_dir=OUT_DIR):
    os.makedirs(out_dir, exist_ok=True)
    gdf.to_file(os.path.join(out_dir, f"{name}.gpkg"), driver="GPKG", engine="fiona", index=False)
    gdf.to_file(os.path.join(out_dir, f"{name}.geojson"), driver="GeoJSON", engine="fiona")
    gdf.drop(columns="geometry").to_csv(os.path.join(out_dir, f"{name}.csv"), index=False)

# Calculate solar capacity and congestion ratio
def calculate_solar_capacity_and_congestion(candidates_df, total_grid_capacity_kw=TOTAL_GRID_CAPACITY_KW):
    """
    Calculate solar capacity and congestion ratio for each candidate building
    
    Args:
        candidates_df: DataFrame with building data including area_m2
        total_grid_capacity_kw: Total grid capacity in kW
    
    Returns:
        DataFrame with additional columns for solar analysis
    """
    df = candidates_df.copy()
    
    # Calculate potential solar capacity (in kW)
    # Assuming 200W per m² of solar panels (0.2 kW/m²)
    SOLAR_POWER_DENSITY_KW_PER_M2 = 0.2
    
    df["solar_capacity_kw"] = df["panel_m2"] * SOLAR_POWER_DENSITY_KW_PER_M2
    
    # Calculate congestion ratio (solar capacity / total grid capacity)
    df["congestion_ratio"] = df["solar_capacity_kw"] / total_grid_capacity_kw
    
    # Calculate congestion percentage
    df["congestion_percentage"] = df["congestion_ratio"] * 100
    
    # Add a priority score based on capacity and congestion impact
    df["solar_priority_score"] = (df["solar_capacity_kw"] / df["solar_capacity_kw"].max() * 100).round(2)
    
    return df

# Column execution
leuven = get_leuven_boundary()
buildings_raw = fetch_buildings_wfs(leuven, GRB_WFS_URL, GRB_WFS_LAYER)
buildings = add_area_centroid(clean_columns(buildings_raw))

candidates = buildings[buildings["area_m2"] >= LARGE_ROOF_MIN_M2].copy()

candidates["panel_m2"] = candidates["area_m2"] * FILL_FACTOR
candidates["kwh_year"] = candidates["panel_m2"] * KWH_PER_M2
candidates["co2_tons"] = candidates["kwh_year"] * CO2_KG_PER_KWH / 1000.0
candidates["rank"] = candidates["co2_tons"].rank(ascending=False, method="dense").astype(int)
candidates = calculate_solar_capacity_and_congestion(candidates)

top200 = candidates.sort_values("co2_tons", ascending=False).head(200).copy()

# Display summary of congestion analysis
print("\n" + "="*60)
print("CONGESTION ANALYSIS SUMMARY")
print("="*60)
print(f"Total grid capacity: {TOTAL_GRID_CAPACITY_KW:,} kW")
print(f"Number of candidate buildings: {len(candidates)}")
print(f"Total potential solar capacity: {candidates['solar_capacity_kw'].sum():,.2f} kW")
print(f"Average congestion ratio per building: {candidates['congestion_ratio'].mean():.6f}")
print(f"Average congestion percentage per building: {candidates['congestion_percentage'].mean():.6f}%")
print(f"Maximum individual congestion: {candidates['congestion_percentage'].max():.6f}%")

# Display top 10 buildings by congestion impact
print("\n" + "="*60)
print("TOP 10 BUILDINGS BY CONGESTION IMPACT")
print("="*60)
top_congestion = candidates.nlargest(10, 'congestion_percentage')[['src_id', 'area_m2', 'solar_capacity_kw', 'congestion_percentage', 'solar_priority_score']]
print(top_congestion.to_string(index=False))

# Save the enhanced candidates data
save_layers(candidates, "candidates_with_congestion")

print(f"\nEnhanced candidates data saved with congestion analysis!")
print(f"New columns added:")
print(f"- solar_capacity_kw: Potential solar capacity in kW")
print(f"- congestion_ratio: Solar capacity / Grid capacity")
print(f"- congestion_percentage: Congestion ratio as percentage")
print(f"- solar_priority_score: Priority score for solar installation")

candidates

Service title: WFS Gebouwenregister
- Gebouwenregister:Gebouw :: Gebouw
- Gebouwenregister:Gebouweenheid :: Gebouweenheid

CONGESTION ANALYSIS SUMMARY
Total grid capacity: 130,000 kW
Number of candidate buildings: 232
Total potential solar capacity: 52,243.94 kW
Average congestion ratio per building: 0.001732
Average congestion percentage per building: 0.173223%
Maximum individual congestion: 5.933328%

TOP 10 BUILDINGS BY CONGESTION IMPACT
         src_id      area_m2  solar_capacity_kw  congestion_percentage  solar_priority_score
Gebouw.20591951 64277.724339        7713.326921               5.933328                100.00
Gebouw.19740151 19750.471188        2370.056543               1.823120                 30.73
Gebouw.16580848 11342.750736        1361.130088               1.047023                 17.65
Gebouw.19697068  8345.700857        1001.484103               0.770372                 12.98
Gebouw.14572288  8147.909898         977.749188               0.752115                 12.

,src_id,wfs_id,object_id,VersieId,GeometrieMethode,GebouwStatus,geometry,area_m2,centroid_x,centroid_y,panel_m2,kwh_year,co2_tons,rank,solar_capacity_kw,congestion_ratio,congestion_percentage,solar_priority_score
180,Gebouw.16580848,https://data.vlaanderen.be/id/gebouw/16580848,16580848,2023-11-03 20:01:35,IngemetenGRB,Gerealiseerd,"POLYGON ((170566.247 181683.711, 170563.079 18...",11342.750736,170512.024489,181694.909534,6805.650442,1.361130e+06,313.059920,3,1361.130088,0.010470,1.047023,17.65
256,Gebouw.31592581,https://data.vlaanderen.be/id/gebouw/31592581,31592581,2025-06-13 08:43:31,IngemetenGRB,Gerealiseerd,"POLYGON ((175426.517 174562.727, 175425.676 17...",502.544103,175408.502967,174566.412367,301.526462,6.030529e+04,13.870217,230,60.305292,0.000464,0.046389,0.78
267,Gebouw.14066971,https://data.vlaanderen.be/id/gebouw/14066971,14066971,2023-11-02 17:01:30,IngemetenGRB,Gehistoreerd,"POLYGON ((172858.039 172983.715, 172843.314 17...",1093.768753,172834.643165,172984.155251,656.261252,1.312523e+05,30.188018,101,131.252250,0.001010,0.100963,1.70
280,Gebouw.20591951,https://data.vlaanderen.be/id/gebouw/20591951,20591951,2025-06-17 05:23:05,IngemetenGRB,Gerealiseerd,"POLYGON ((171605.231 174511.563, 171605.597 17...",64277.724339,171407.692197,174258.310655,38566.634603,7.713327e+06,1774.065192,1,7713.326921,0.059333,5.933328,100.00
389,Gebouw.14174954,https://data.vlaanderen.be/id/gebouw/14174954,14174954,2023-11-02 16:10:50,IngemetenGRB,Gerealiseerd,"POLYGON ((171735.803 172812.609, 171734.307 17...",1396.887892,171740.653156,172791.875141,838.132735,1.676265e+05,38.554106,77,167.626547,0.001289,0.128943,2.17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9859,Gebouw.16830557,https://data.vlaanderen.be/id/gebouw/16830557,16830557,2023-11-03 10:16:40,IngemetenGRB,Gerealiseerd,"POLYGON ((170328.574 181572.873, 170324.817 18...",853.301802,170302.265104,181571.059617,511.981081,1.023962e+05,23.551130,132,102.396216,0.000788,0.078766,1.33
9909,Gebouw.19741568,https://data.vlaanderen.be/id/gebouw/19741568,19741568,2023-11-03 04:57:48,IngemetenGRB,Gerealiseerd,"POLYGON ((174609.033 175166.101, 174602.758 17...",664.793458,174608.985128,175151.660670,398.876075,7.977522e+04,18.348299,176,79.775215,0.000614,0.061366,1.03
9937,Gebouw.12256261,https://data.vlaanderen.be/id/gebouw/12256261,12256261,2023-11-03 20:18:21,IngemetenGRB,Gerealiseerd,"POLYGON ((177260.381 168797.607, 177263.63 168...",771.918343,177248.426247,168785.735717,463.151006,9.263020e+04,21.304946,145,92.630201,0.000713,0.071254,1.20
9972,Gebouw.14571733,https://data.vlaanderen.be/id/gebouw/14571733,14571733,2023-11-02 15:44:34,IngemetenGRB,Gerealiseerd,"POLYGON ((174761.029 171105.835, 174763.596 17...",4160.738713,174735.386578,171130.319673,2496.443228,4.992886e+05,114.836388,16,499.288646,0.003841,0.384068,6.47
